## Libraries

In [1]:
import random
import pickle

# Data manipulation
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# models
import sklearn
import sktime
import keras
import statsmodels.api as sm
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from tensorflow import keras
from tensorflow.keras import layers

# model selection and metrics
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import RFE

# constants
PROCESSED_DATA_PATH = '../data/processed-data/'
MODEL_PATH = '../models/trained-models/'
TARGET_COLS = ['target1', 'target2', 'target3', 'target4']
TEST_SPLIT_DATE = '2021-04-30'

# disable warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'

## Auxiliary Functions

In [3]:
def plot_time_series(df, groupby = 'median', cols = TARGET_COLS):
    df_melted = df.groupby('Dt').median()[cols]\
    .melt(var_name='target', value_name='value', ignore_index=False)
    df_melted.reset_index(inplace=True)

    sns.set(rc={'figure.figsize':(10,20)})
    sns.FacetGrid(df_melted, col='target', col_wrap=2, height=9,aspect=2, sharey=False)\
    .map(sns.lineplot, 'Dt', 'value');
    
def plot_train_test_pred(train, test, pred = pd.DataFrame(), groupby = 'median', cols = TARGET_COLS):
    train['type'] = 'train'
    test['type'] = 'test'
    pred['type'] = 'pred'
    
    cols.append('type')

    df = pd.concat([train, test, pred], axis=0)
    df_melted = df.groupby('Dt').median()[cols]\
        .melt(var_name='target', value_name='value', ignore_index=False)
    df_melted.reset_index(inplace=True)
    
    sns.set(rc={'figure.figsize':(10,20)})
    sns.FacetGrid(df_melted, col='target', col_wrap=2, height=9,aspect=2, sharey=False)\
        .map(sns.lineplot, 'Dt', 'value', hue='type');

## Data Prep

In [17]:
df = pd.read_pickle(PROCESSED_DATA_PATH + 'shifted_targets.pkl')

In [5]:
#train = df[(df.Dt <= "2021-01-31") & (df.Dt >= "2018-01-01")] 
#val = df[(df.Dt <= "2021-04-30") & (df.Dt >= "2021-02-01")] 
#test = df[(df.Dt <= "2021-07-31") & (df.Dt >= "2021-05-01")]

#train.to_csv('train.csv', index=None)
#val.to_csv('validation.csv', index=None) 
#test.to_csv('test.csv', index=None)

train = pd.read_csv('train.csv')
val = pd.read_csv('validation.csv')
test = pd.read_csv('test.csv')

## Baselines

In [7]:
summary = pd.DataFrame(columns=TARGET_COLS,index=['Média','Média por Jogador','Mediana','Mediana por Jogador','Naive'])
# train + validation dataset
train_val = pd.concat([train, val], axis=0)

In [13]:
train.shape[0] + val.shape[0] == train_val.shape[0]

True

In [14]:
media = train_val[TARGET_COLS].mean()
media_por_jogador = train_val.groupby('IdPlayer')[TARGET_COLS].mean()
mediana = train_val[TARGET_COLS].median()
mediana_por_jogador = train_val.groupby('IdPlayer')[TARGET_COLS].median()
naive = train_val[train_val['Dt']=='2021-04-30'].set_index('IdPlayer')[TARGET_COLS]

In [15]:
for target in TARGET_COLS:
    
    y_true = test[target]
    
    mediapj_pred = test['IdPlayer'].map(media_por_jogador[target].to_dict())
    medianapj_pred = test['IdPlayer'].map(mediana_por_jogador[target].to_dict())
    naive_pred = test['IdPlayer'].map(naive[target].to_dict())
    
    mediana_pred = [mediana[target] for i in test.index]
    media_pred = [media[target] for i in test.index]
    
    summary.loc['Média',target]  = mean_absolute_error(y_true,media_pred)
    summary.loc['Média por Jogador',target]  = mean_absolute_error(y_true,mediapj_pred)
    summary.loc['Mediana',target]  = mean_absolute_error(y_true,mediana_pred)
    summary.loc['Mediana por Jogador',target]  = mean_absolute_error(y_true,medianapj_pred)
    summary.loc['Naive',target]  = mean_absolute_error(y_true,naive_pred)
    
summary['Average'] = summary.mean(axis=1)

In [16]:
summary

,target1,target2,target3,target4,Average
Média,1.126843,2.739029,1.068968,1.477766,1.603152
Média por Jogador,0.939999,2.251019,0.9543,1.025011,1.292582
Mediana,0.712801,1.651943,0.498075,1.139852,1.000668
Mediana por Jogador,0.702606,1.56062,0.493126,0.925954,0.920577
Naive,1.168903,1.808041,0.761283,1.520494,1.314680


## LASSO and Multitask LASSO

In [18]:
# dropping the columns with a high shift
for i in range(4):
    for shift in [6, 7, 14, 30]:
        df.drop(f'target{i+1}_shift_{shift}', axis=1, inplace=True)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2695788 entries, 0 to 2695787
Data columns (total 27 columns):
 #   Column           Dtype         
---  ------           -----         
 0   Dt               datetime64[ns]
 1   IdPlayer         int64         
 2   target1          float32       
 3   target2          float32       
 4   target3          float32       
 5   target4          float32       
 6   IdDtPlayer       object        
 7   target1_shift_1  float32       
 8   target2_shift_1  float32       
 9   target3_shift_1  float32       
 10  target4_shift_1  float32       
 11  target1_shift_2  float32       
 12  target2_shift_2  float32       
 13  target3_shift_2  float32       
 14  target4_shift_2  float32       
 15  target1_shift_3  float32       
 16  target2_shift_3  float32       
 17  target3_shift_3  float32       
 18  target4_shift_3  float32       
 19  target1_shift_4  float32       
 20  target2_shift_4  float32       
 21  target3_shift_4  float32       

In [21]:
df_playerBoxScores = pd.read_pickle(PROCESSED_DATA_PATH + 'playerBoxScores.pkl')
df_playerBoxScores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 219727 entries, 0 to 451
Data columns (total 54 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   IdGame                    219727 non-null  int64  
 1   DtGame                    219727 non-null  object 
 2   DtGameUTC                 219727 non-null  object 
 3   IdPlayer                  219727 non-null  int64  
 4   IdTeam                    219727 non-null  int64  
 5   NuJersey                  219690 non-null  object 
 6   CdPosition                219727 non-null  int64  
 7   NuStrikeOutsPitching      65466 non-null   float64
 8   NuBattingOrder            183390 non-null  float64
 9   NuGamesPlayedBatting      183395 non-null  float64
 10  NuFlyOuts                 183395 non-null  float64
 11  NuGroundOuts              183395 non-null  float64
 12  NuRunsScored              183395 non-null  float64
 13  NuDoubles                 183395 non-null  floa

In [24]:
df_join = pd.merge(df, df_playerBoxScores, on=['IdDtPlayer'], how='left')
df_join.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2698457 entries, 0 to 2698456
Data columns (total 80 columns):
 #   Column                    Non-Null Count    Dtype         
---  ------                    --------------    -----         
 0   Dt                        2698457 non-null  datetime64[ns]
 1   IdPlayer_x                2698457 non-null  int64         
 2   target1                   2698457 non-null  float32       
 3   target2                   2698457 non-null  float32       
 4   target3                   2698457 non-null  float32       
 5   target4                   2698457 non-null  float32       
 6   IdDtPlayer                2698457 non-null  object        
 7   target1_shift_1           2696396 non-null  float32       
 8   target2_shift_1           2696396 non-null  float32       
 9   target3_shift_1           2696396 non-null  float32       
 10  target4_shift_1           2696396 non-null  float32       
 11  target1_shift_2           2694335 non-null  float3

In [25]:
f = [c for c in df_join.columns if c not in ['IdGame',
                                              'DtGame',
                                              'DtGameUTC',
                                              'IdPlayer_y',
                                              'IdTeam',
                                              'NuJersey',
                                              'CdPosition', 
                                              'target1_shift_1', 
                                              'target2_shift_1',
                                              'target3_shift_1',
                                              'target1_shift_2',
                                              'target3_shift_2',
                                              'target4_shift_2',
                                              'target1_shift_3',
                                              'target2_shift_3',
                                              'target3_shift_3',
                                              'target4_shift_3',
                                              'target1_shift_4',
                                              'target2_shift_4',
                                              'target3_shift_4',
                                              'target4_shift_4',
                                              'target1_shift_5',
                                              'target2_shift_5',
                                              'target3_shift_5',
                                              'target4_shift_5']]

df_join[f] = df_join[f].fillna(0)                      

In [26]:
df_join = df_join.dropna(subset=[             
    'target1_shift_1', 
    'target2_shift_1',
    'target3_shift_1',
    'target1_shift_2',
    'target3_shift_2',
    'target4_shift_2',
    'target1_shift_3',
    'target2_shift_3',
    'target3_shift_3',
    'target4_shift_3',
    'target1_shift_4',
    'target2_shift_4',
    'target3_shift_4',
    'target4_shift_4',
    'target1_shift_5',
    'target3_shift_5',
    'target4_shift_5',
    'CdPosition'])

In [27]:
df_join.isna().any()[lambda x: x]

NuJersey    True
dtype: bool